In [1]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.0+cpu.html
!pip install --upgrade pip setuptools wheel
!pip install --upgrade torch torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.5.0+cpu.html


In [2]:
pip install certifi

In [3]:
import torch
import torch_cluster
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec, PNAConv
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.utils import degree

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0].to(device)

Processing...
Done!


In [5]:
# Compute node degrees for PNAConv
deg = degree(data.edge_index[0], data.num_nodes).to(device)

# Define aggregators and scalers for PNAConv
aggregators = ['mean', 'min', 'max', 'std']
scalers = ['identity', 'amplification', 'attenuation']

hidden_dim = 64

In [6]:
class PNAModelStandard(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, aggregators, scalers, deg):
        super(PNAModelStandard, self).__init__()
        self.conv = PNAConv(in_channels, hidden_channels, aggregators=aggregators,
                            scalers=scalers, deg=deg)
        self.lin = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # PNA aggregation step
        h = self.conv(x, edge_index)
        h = F.relu(h)
        h = self.lin(h)
        return h

model = PNAModelStandard(
    in_channels=dataset.num_node_features,
    hidden_channels=hidden_dim,
    out_channels=dataset.num_classes,
    aggregators=aggregators,
    scalers=scalers,
    deg=deg
).to(device)

In [7]:
# Setup training components
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

x = data.x

In [8]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    out = model(x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # [train_acc, val_acc, test_acc]

In [10]:
# Train and evaluate
best_val_acc = 0
test_acc_at_best_val = 0

print("Training PNA model...")
for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc_at_best_val = test_acc

    if epoch % 20 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, "
              f"Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")

print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")
print(f"Test Accuracy at Best Val: {test_acc_at_best_val:.4f}")

Training PNA model...
Epoch: 020, Loss: 0.0012, Train: 1.0000, Val: 0.5740, Test: 0.5660
Epoch: 040, Loss: 0.0015, Train: 1.0000, Val: 0.5900, Test: 0.5810
Epoch: 060, Loss: 0.0018, Train: 1.0000, Val: 0.6000, Test: 0.6000
Epoch: 080, Loss: 0.0021, Train: 1.0000, Val: 0.6000, Test: 0.6130
Epoch: 100, Loss: 0.0024, Train: 1.0000, Val: 0.6120, Test: 0.6130
Epoch: 120, Loss: 0.0025, Train: 1.0000, Val: 0.6200, Test: 0.6250
Epoch: 140, Loss: 0.0026, Train: 1.0000, Val: 0.6220, Test: 0.6330
Epoch: 160, Loss: 0.0027, Train: 1.0000, Val: 0.6200, Test: 0.6420
Epoch: 180, Loss: 0.0028, Train: 1.0000, Val: 0.6280, Test: 0.6460
Epoch: 200, Loss: 0.0028, Train: 1.0000, Val: 0.6300, Test: 0.6450

Best Validation Accuracy: 0.6320
Test Accuracy at Best Val: 0.6440
